In [1]:
import random
import numpy as np
from docplex.mp.model import Model
import networkx as nx
from itertools import islice

edges = [
    # From Başlangıç nodes
    ("Başlangıç1", "Ara10"),
    ("Başlangıç2", "Ara11"),
    ("Başlangıç3", "Ara13"),
    ("Başlangıç3", "Ara12"),
    ("Başlangıç3", "Başlangıç4"),
    ("Başlangıç4", "Başlangıç3"),
    ("Başlangıç4", "Başlangıç5"),
    ("Başlangıç5", "Ara13"),
    ("Başlangıç5", "Başlangıç4"),
    ("Başlangıç6", "Ara14"),
    ("Başlangıç6", "Başlangıç7"),
    ("Başlangıç7", "Başlangıç6"),
    ("Başlangıç8", "Ara21"),
    ("Başlangıç9", "Ara24"),

    # Intermediate connections
    ("Ara10", "Ara16"),
    ("Ara11", "Ara10"),
    ("Ara11", "Ara12"),
    ("Ara11", "Ara17"),
    ("Ara12", "Ara18"),
    ("Ara13", "Ara14"),
    ("Ara13", "Ara19"),
    ("Ara14", "Ara15"),
    ("Ara15", "Ara20"),
    ("Ara15", "Ara35"),
    ("Ara16", "Ara28"),
    ("Ara17", "Ara16"),
    ("Ara17", "Ara18"),
    ("Ara18", "Ara19"),
    ("Ara19", "Ara20"),
    ("Ara20", "Ara34"),
    ("Ara21", "Ara22"),
    ("Ara21", "Ara35"),
    ("Ara22", "Ara23"),
    ("Ara22", "Bit45"),
    ("Ara23", "Bit46"),
    ("Ara24", "Ara25"),
    ("Ara24", "Ara36"),
    ("Ara25", "Ara26"),
    ("Ara25", "Ara37"),
    ("Ara26", "Ara27"),
    ("Ara26", "Ara38"),
    ("Ara27", "Ara39"),
    ("Ara28", "Bit29"),
    ("Ara28", "Bit50"),
    ("Ara32", "Bit31"),
    ("Ara32", "Bit40"),
    ("Ara33", "Ara32"),
    ("Ara33", "Bit41"),
    ("Ara34", "Bit42"),
    ("Ara34", "Ara33"),
    ("Ara35", "Ara34"),
    ("Ara35", "Bit44"),
    ("Ara36", "Ara37"),
    ("Ara36", "Bit47"),
    ("Ara37", "Ara38"),
    ("Ara37", "Bit48"),
    ("Ara38", "Ara39"),
    ("Ara39", "Bit49"),
]

travel_time = {
    # From Başlangıç nodes
    ("Başlangıç1", "Ara10"): 32.4,
    ("Başlangıç2", "Ara11"): 18,
    ("Başlangıç3", "Ara13"): 21.6,
    ("Başlangıç3", "Ara12"): 19.8,
    ("Başlangıç3", "Başlangıç4"): 0,
    ("Başlangıç4", "Başlangıç3"): 0,
    ("Başlangıç4", "Başlangıç5"): 0,
    ("Başlangıç5", "Ara13"): 37.8,
    ("Başlangıç5", "Başlangıç4"): 0,
    ("Başlangıç6", "Ara14"): 27,
    ("Başlangıç6", "Başlangıç7"): 0,
    ("Başlangıç7", "Başlangıç6"): 0,
    ("Başlangıç8", "Ara21"): 63,
    ("Başlangıç9", "Ara24"): 34.2,

    # Intermediate connections
    ("Ara10", "Ara16"): 43.2,
    ("Ara11", "Ara10"): 77.4,
    ("Ara11", "Ara12"): 68,
    ("Ara11", "Ara17"): 18,
    ("Ara12", "Ara18"): 27,
    ("Ara13", "Ara14"): 54,
    ("Ara13", "Ara19"): 34.2,
    ("Ara14", "Ara15"): 10.8,
    ("Ara15", "Ara20"): 21.6,
    ("Ara15", "Ara35"): 55.8,
    ("Ara16", "Ara28"): 176.4,
    ("Ara17", "Ara16"): 14,
    ("Ara17", "Ara18"): 12,
    ("Ara18", "Ara19"): 27,
    ("Ara19", "Ara20"): 66.6,
    ("Ara20", "Ara34"): 46.8,
    ("Ara21", "Ara22"): 41.4,
    ("Ara21", "Ara35"): 18,
    ("Ara22", "Ara23"): 32.4,
    ("Ara22", "Bit45"): 81,
    ("Ara23", "Bit46"): 77.4,
    ("Ara24", "Ara25"): 21.6,
    ("Ara24", "Ara36"): 25.2,
    ("Ara25", "Ara26"): 52.2,
    ("Ara25", "Ara37"): 25.2,
    ("Ara26", "Ara27"): 48.6,
    ("Ara26", "Ara38"): 25.2,
    ("Ara27", "Ara39"): 28.8,
    ("Ara28", "Bit29"): 50.4,
    ("Ara28", "Bit50"): 45,
    ("Ara32", "Bit31"): 19.8,
    ("Ara32", "Bit40"): 27,
    ("Ara33", "Ara32"): 10.8,
    ("Ara33", "Bit41"): 32.4,
    ("Ara34", "Bit42"): 45,
    ("Ara34", "Ara33"): 63,
    ("Ara35", "Ara34"): 48.6,
    ("Ara35", "Bit44"): 100.8,
    ("Ara36", "Ara37"): 21.6,
    ("Ara36", "Bit47"): 70.2,
    ("Ara37", "Ara38"): 18,
    ("Ara37", "Bit48"): 25,
    ("Ara38", "Ara39"): 50.4,
    ("Ara39", "Bit49"): 52.2,
}

capacity = {
    # From Başlangıç nodes
    ("Başlangıç1", "Ara10"): 4400,
    ("Başlangıç2", "Ara11"): 2200,
    ("Başlangıç3", "Ara13"): 2200,
    ("Başlangıç3", "Ara12"): 6600,
    ("Başlangıç3", "Başlangıç4"): 100000,
    ("Başlangıç4", "Başlangıç3"): 100000,
    ("Başlangıç4", "Başlangıç5"): 100000,
    ("Başlangıç5", "Ara13"): 4400,
    ("Başlangıç5", "Başlangıç4"): 100000,
    ("Başlangıç6", "Ara14"): 6600,
    ("Başlangıç6", "Başlangıç7"): 100000,
    ("Başlangıç7", "Başlangıç6"): 100000,
    ("Başlangıç8", "Ara21"): 4400,
    ("Başlangıç9", "Ara24"): 6600,

    # Intermediate connections
    ("Ara10", "Ara16"): 4400,
    ("Ara11", "Ara10"): 4400,
    ("Ara11", "Ara12"): 4400,
    ("Ara11", "Ara17"): 2200,
    ("Ara12", "Ara18"): 2200,
    ("Ara13", "Ara14"): 6600,
    ("Ara13", "Ara19"): 4400,
    ("Ara14", "Ara15"): 8800,
    ("Ara15", "Ara20"): 4400,
    ("Ara15", "Ara35"): 6600,
    ("Ara16", "Ara28"): 6600,
    ("Ara17", "Ara16"): 4400,
    ("Ara17", "Ara18"): 6600,
    ("Ara18", "Ara19"): 6600,
    ("Ara19", "Ara20"): 6600,
    ("Ara20", "Ara34"): 8800,
    ("Ara21", "Ara22"): 6600,
    ("Ara21", "Ara35"): 4400,
    ("Ara22", "Ara23"): 6600,
    ("Ara22", "Bit45"): 2200,
    ("Ara23", "Bit46"): 4400,
    ("Ara24", "Ara25"): 2200,
    ("Ara24", "Ara36"): 6600,
    ("Ara25", "Ara26"): 4400,
    ("Ara25", "Ara37"): 6600,
    ("Ara26", "Ara27"): 4400,
    ("Ara26", "Ara38"): 4400,
    ("Ara27", "Ara39"): 4400,
    ("Ara28", "Bit29"): 4400,
    ("Ara28", "Bit50"): 4400,
    ("Ara32", "Bit31"): 2200,
    ("Ara32", "Bit40"): 2200,
    ("Ara33", "Ara32"): 4400,
    ("Ara33", "Bit41"): 2200,
    ("Ara34", "Bit42"): 4400,
    ("Ara34", "Ara33"): 6600,
    ("Ara35", "Ara34"): 6600,
    ("Ara35", "Bit44"): 6600,
    ("Ara36", "Ara37"): 6600,
    ("Ara36", "Bit47"): 2200,
    ("Ara37", "Ara38"): 2200,
    ("Ara37", "Bit48"): 2200,
    ("Ara38", "Ara39"): 2200,
    ("Ara39", "Bit49"): 4400,
}

passengers = {
    "Başlangıç1": 2000,
    "Başlangıç2": 1000,
    "Başlangıç3": 2200,
    "Başlangıç4": 0,
    "Başlangıç5": 2800,
    "Başlangıç6": 3000,
    "Başlangıç7": 0,
    "Başlangıç8": 2000,
    "Başlangıç9": 3000,

    "Ara10": 0, "Ara11": 0, "Ara12": 0, "Ara13": 0, "Ara14": 0,
    "Ara15": 0, "Ara16": 0, "Ara17": 0, "Ara18": 0, "Ara19": 0,
    "Ara20": 0, "Ara21": 0, "Ara22": 0, "Ara23": 0, "Ara24": 0,
    "Ara25": 0, "Ara26": 0, "Ara27": 0, "Ara28": 0, "Bit29": 0,
    "Bit30": 0, "Bit31": 0, "Ara32": 0, "Ara33": 0, "Ara34": 0,
    "Ara35": 0, "Ara36": 0, "Ara37": 0, "Ara38": 0, "Ara39": 0,
    "Bit40": 0, "Bit41": 0, "Bit42": 0, "Bit43": 0, "Bit44": 0,
    "Bit45": 0, "Bit46": 0, "Bit47": 0, "Bit48": 0, "Bit49": 0,
    "Bit50": 0
}

In [2]:
def create_stage_two_network(original_total_passengers):
    # Yeni ağ yapısı
    new_edges = [
        ('Başlangıç51', 'Başlangıç52'), ('Başlangıç51', 'Ara64'), 
        ('Başlangıç52', 'Başlangıç51'), ('Başlangıç52', 'Başlangıç56'), ('Başlangıç52', 'Ara64'),
        ('Başlangıç56', 'Başlangıç57'), ('Başlangıç57', 'Başlangıç56'), ('Başlangıç56', 'Ara65'),
        ('Başlangıç57', 'Başlangıç67'), ('Başlangıç57', 'Başlangıç56'), ('Başlangıç57', 'Ara66'),
        ('Başlangıç67', 'Başlangıç57'), ('Başlangıç67', 'Başlangıç68'), ('Başlangıç67', 'Ara76'),
        ('Başlangıç68', 'Başlangıç67'), ('Başlangıç68', 'Başlangıç70'), ('Başlangıç68', 'Ara77'),
        ('Başlangıç70', 'Başlangıç68'), ('Başlangıç70', 'Ara78'),
        ('Ara64', 'Ara65'), ('Ara64', 'Ara74'),
        ('Ara65', 'Ara66'), ('Ara65', 'Ara72'),
        ('Ara66', 'Ara73'), 
        ('Ara72', 'Ara73'), ('Ara72', 'Ara74'),
        ('Ara73', 'Ara75'),
        ('Ara74', 'Ara75'),
        ('Ara75', 'Ara76'), ('Ara75', 'Bit79'),
        ('Ara76', 'Ara77'), ('Ara76', 'Bit80'),
        ('Ara77', 'Ara78'),
        ('Ara78', 'Bit81'),
    ]
    
    # Yeni seyahat süreleri
    new_travel_time = {
        ('Başlangıç51', 'Başlangıç52'):0, ('Başlangıç51', 'Ara64'):90, 
        ('Başlangıç52', 'Başlangıç51'):0, ('Başlangıç52', 'Başlangıç56'):0, ('Başlangıç52', 'Ara64'): 36,
        ('Başlangıç56', 'Başlangıç57'):0, ('Başlangıç57', 'Başlangıç56'):0, ('Başlangıç56', 'Ara65'): 27,
        ('Başlangıç57', 'Başlangıç67'):0, ('Başlangıç57', 'Başlangıç56'):0, ('Başlangıç57', 'Ara66'): 34.20,
        ('Başlangıç67', 'Başlangıç57'):0, ('Başlangıç67', 'Başlangıç68'):0, ('Başlangıç67', 'Ara76'): 50.4,
        ('Başlangıç68', 'Başlangıç67'):0, ('Başlangıç68', 'Başlangıç70'):0, ('Başlangıç68', 'Ara77'): 68.4,
        ('Başlangıç70', 'Başlangıç68'):0, ('Başlangıç70', 'Ara78'): 72.00,
        ('Ara64', 'Ara65'): 86.4, ('Ara64', 'Ara74'): 90,
        ('Ara65', 'Ara66'): 79.2, ('Ara65', 'Ara72'): 14.4,
        ('Ara66', 'Ara73'): 23.4, 
        ('Ara72', 'Ara73'): 79.2, ('Ara72', 'Ara74'): 14.4,
        ('Ara73', 'Ara75'): 23.4,
        ('Ara74', 'Ara75'): 79.2,
        ('Ara75', 'Ara76'): 52.2, ('Ara75', 'Bit79'): 39.6,
        ('Ara76', 'Ara77'): 73.8, ('Ara76', 'Bit80'): 55.8,
        ('Ara77', 'Ara78'): 32.4,
        ('Ara78', 'Bit81'): 79.2,
    }
    
    # Yeni kapasiteler
    new_capacity = {
        ('Başlangıç51', 'Başlangıç52'):1000000, ('Başlangıç51', 'Ara64'):2200, 
        ('Başlangıç52', 'Başlangıç51'):1000000, ('Başlangıç52', 'Başlangıç56'):1000000, ('Başlangıç52', 'Ara64'): 4400,
        ('Başlangıç56', 'Başlangıç57'):1000000, ('Başlangıç57', 'Başlangıç56'):1000000, ('Başlangıç56', 'Ara65'): 6600,
        ('Başlangıç57', 'Başlangıç67'):1000000, ('Başlangıç57', 'Başlangıç56'):1000000, ('Başlangıç57', 'Ara66'): 4400,
        ('Başlangıç67', 'Başlangıç57'):1000000, ('Başlangıç67', 'Başlangıç68'):1000000, ('Başlangıç67', 'Ara76'): 6600,
        ('Başlangıç68', 'Başlangıç67'):1000000, ('Başlangıç68', 'Başlangıç70'):1000000, ('Başlangıç68', 'Ara77'): 2200,
        ('Başlangıç70', 'Başlangıç68'):1000000, ('Başlangıç70', 'Ara78'): 6600,
        ('Ara64', 'Ara65'): 4400, ('Ara64', 'Ara74'): 4400,
        ('Ara65', 'Ara66'): 13200, ('Ara65', 'Ara72'): 6600,
        ('Ara66', 'Ara73'): 8800, 
        ('Ara72', 'Ara73'): 2200, ('Ara72', 'Ara74'): 4400,
        ('Ara73', 'Ara75'): 11000,
        ('Ara74', 'Ara75'): 6600,
        ('Ara75', 'Ara76'): 6600, ('Ara75', 'Bit79'): 11000,
        ('Ara76', 'Ara77'): 2200, ('Ara76', 'Bit80'): 11000,
        ('Ara77', 'Ara78'): 4400,
        ('Ara78', 'Bit81'): 11000,
    }
    
    # Yeni yolcu dağılımı - Daha güvenli yöntem
    baslangic_noktalari = ['Başlangıç51', 'Başlangıç52', 'Başlangıç56', 'Başlangıç57', 'Başlangıç67', 'Başlangıç68', 'Başlangıç70']
    new_passengers = {}
    
    # Rastgele oranlar oluştur
    oranlar = np.random.dirichlet(np.ones(len(baslangic_noktalari)))
    
    # Toplam yolcu sayısını oranlara göre dağıt
    for i, nokta in enumerate(baslangic_noktalari):
        new_passengers[nokta] = int(original_total_passengers * oranlar[i])
    
    # Yuvarlama hatalarını düzelt
    toplam = sum(new_passengers.values())
    if toplam != original_total_passengers:
        fark = original_total_passengers - toplam
        new_passengers[baslangic_noktalari[0]] += fark
    
    return new_edges, new_travel_time, new_capacity, new_passengers


In [5]:
import random
import numpy as np
from docplex.mp.model import Model
import networkx as nx
from itertools import islice



def solve_model(model):
    model.parameters.emphasis.mip = 1               
    model.parameters.mip.strategy.search = 1
    model.parameters.timelimit = 60                 
    model.parameters.mip.tolerances.mipgap = 0.005   
    solution = model.solve()
    return solution


def create_evacuation_model(edges, passengers, capacity, travel_time):
    
    nodes = set()
    for edge in edges:
        nodes.add(edge[0])
        nodes.add(edge[1])
    
    baslangic_nodes = [node for node in passengers.keys() if node.startswith("Başlangıç") and passengers[node] > 0]
    bitis_nodes = list(set([node for edge in edges for node in edge if node.startswith("Bit")]))
    ara_nodes = [node for node in passengers.keys() if node.startswith("Ara")]
    
    toplam_baslangic_yolcu = sum(passengers[s] for s in baslangic_nodes)
    print(f"Başlangıç noktalarındaki toplam yolcu: {toplam_baslangic_yolcu}")
    
    G = nx.DiGraph()
    for (i, j), time in travel_time.items():
        G.add_edge(i, j, weight=time)
    
    K = 100  
    paths = {}
    path_time = {}
    path_length = {}
    
    def k_shortest_paths(G, source, target, k):
        return list(islice(nx.shortest_simple_paths(G, source, target, weight='weight'), k))
    
    for s in baslangic_nodes:
        for t in bitis_nodes:
            try:
                paths[(s, t)] = k_shortest_paths(G, s, t, K)
                for k, path in enumerate(paths[(s, t)]):
                    total_time = sum(travel_time.get((path[i], path[i+1]), 0) 
                                     for i in range(len(path)-1))
                    path_time[(s, t, k)] = total_time
                    path_length[(s, t, k)] = len(path) - 1
            except nx.NetworkXNoPath:
                paths[(s, t)] = []
    
    delta = {}
    for s in baslangic_nodes:
        for t in bitis_nodes:
            for k, path in enumerate(paths[(s, t)]):
                delta[(s, t, k)] = {}
                for edge in edges:
                    if len(path) > 1:
                        is_in_path = any(path[i] == edge[0] and path[i+1] == edge[1] 
                                       for i in range(len(path)-1))
                        delta[(s, t, k)][edge] = 1 if is_in_path else 0
    
    model = Model(name='Tahliye_Modeli')
    
    path_flow = {}
    for s in baslangic_nodes:
        for t in bitis_nodes:
            for k in range(len(paths[(s, t)])):
                path_flow[(s, t, k)] = model.continuous_var(
                    name=f'path_flow_{s}_{t}_{k}',
                    lb=0
                )
    
    # Objective function
    model.minimize(
        model.sum(
            path_time[(s, t, k)] * path_flow[(s, t, k)]
            for s in baslangic_nodes
            for t in bitis_nodes
            for k in range(len(paths[(s, t)]))
            if (s, t, k) in path_time
        )
    )
    
    # Constraints
    # 1. 
    for s in baslangic_nodes:
        model.add_constraint(
            model.sum(
                path_flow[(s, t, k)]
                for t in bitis_nodes
                for k in range(len(paths[(s, t)]))
                if (s, t, k) in path_flow
            ) == passengers[s],
            ctname=f'source_flow_{s}'
        )
    
    # 2. 
    for edge in edges:
        model.add_constraint(
            model.sum(
                path_flow[(s, t, k)] * delta[(s, t, k)][edge]
                for s in baslangic_nodes
                for t in bitis_nodes
                for k in range(len(paths[(s, t)]))
                if (s, t, k) in path_flow and (s, t, k) in delta
            ) <= capacity[edge],
            ctname=f'capacity_{edge[0]}_{edge[1]}'
        )
    
    # 3
    model.add_constraint(
        model.sum(
            path_flow[(s, t, k)]
            for s in baslangic_nodes
            for t in bitis_nodes
            for k in range(len(paths[(s, t)]))
            if (s, t, k) in path_flow
        ) == toplam_baslangic_yolcu,
        ctname='total_flow_balance'
    )
    
    return (model, paths, path_flow, path_time, baslangic_nodes, 
            bitis_nodes, passengers, capacity, delta, toplam_baslangic_yolcu, edges)

In [6]:
def print_passenger_distribution(passengers):
    print("\n========= BAŞLANGIÇ NOKTALARI YOLCU DAĞILIMI =========")
    print("-" * 50)
    print(f"{'Başlangıç Noktası':<20} {'Yolcu Sayısı':>15}")
    print("-" * 50)
    
    toplam_yolcu = 0
    for node, count in passengers.items():
        if node.startswith('Başlangıç') and count > 0:
            print(f"{node:<20} {count:>15,}")
            toplam_yolcu += count
    
    print("-" * 50)
    print(f"{'TOPLAM':<20} {toplam_yolcu:>15,}")
    print("-" * 50)
    
    return toplam_yolcu
    
def print_results(solution, path_flow, paths, path_time, baslangic_nodes, 
                  bitis_nodes, passengers, capacity, delta, toplam_baslangic_yolcu, edges):
    """Sonuçları yazdır, kontrolleri yap ve DOCplex çözüm detaylarını göster"""
    
    print("\nSONUÇLAR:")
    print(f"Toplam Seyahat Süresi: {solution.objective_value:.2f} dakika")
    
    # DOCplex çözüm detaylarını yazdırma
    try:
        details = solution.solve_details
        print("\nÇÖZÜM DETAYLARI:")
        print(f"Optimality Gap (Göreceli GAP): {details.mip_relative_gap:.4f}")
        print(f"Çözüm Süresi: {details.time:.2f} saniye")
        print(f"Ziyaret Edilen Düğüm Sayısı: {details.number_of_nodes}")
        print(f"İterasyon Sayısı: {details.number_of_iterations}")
    except Exception as e:
        print("\nÇözüm detayları alınamadı:", e)
    
    print("\nYOL AKIŞLARI:")
    print("-" * 80)
    print(f"{'Başlangıç':15} {'Bitiş':15} {'Yol No':8} {'Akış':10} {'Süre':10}")
    print("-" * 80)
    
    # Bitiş noktalarındaki akışları hesapla
    bitis_yolcu = {t: 0 for t in bitis_nodes}
    baslangic_yolcu_gerceklesen = {s: 0 for s in baslangic_nodes}
    
    for s in baslangic_nodes:
        for t in bitis_nodes:
            for k in range(len(paths[(s, t)])):
                if (s, t, k) in path_flow:
                    flow_val = solution.get_value(path_flow[(s, t, k)])
                    if flow_val > 0.01:
                        bitis_yolcu[t] += flow_val
                        baslangic_yolcu_gerceklesen[s] += flow_val
                        print(f"{s:15} {t:15} {k:8d} {flow_val:10.2f} {path_time[(s, t, k)]:10.2f}")
                        print(f"Yol: {' -> '.join(paths[(s, t)][k])}")
    
    # Kontroller
    print("\nKONTROLLER:")
    print("-" * 50)
    
    # 1. Toplam akış kontrolü
    toplam_gerceklesen_akis = sum(bitis_yolcu.values())
    print(f"Toplam gerçekleşen akış: {toplam_gerceklesen_akis:.2f}")
    print(f"Toplam başlangıç yolcu: {toplam_baslangic_yolcu}")
    
    if abs(toplam_gerceklesen_akis - toplam_baslangic_yolcu) < 0.01:
        print("✓ Toplam akış dengesi sağlandı")
    else:
        print("✗ Toplam akış dengesi sağlanamadı!")
        print(f"   Fark: {abs(toplam_gerceklesen_akis - toplam_baslangic_yolcu):.2f}")
    
    # 2. Başlangıç noktaları kontrolü
    print("\nBAŞLANGIÇ NOKTALARI KONTROLÜ:")
    for s in baslangic_nodes:
        print(f"{s}: Hedef={passengers[s]:.2f}, Gerçekleşen={baslangic_yolcu_gerceklesen[s]:.2f}")
        if abs(passengers[s] - baslangic_yolcu_gerceklesen[s]) > 0.01:
            print(f"✗ {s} için akış dengesi sağlanamadı!")
    
    # 3. Bitiş noktaları dağılımı
    print("\nBİTİŞ NOKTALARI DAĞILIMI:")
    print("-" * 50)
    for t in bitis_nodes:
        print(f"{t}: {bitis_yolcu[t]:.2f} yolcu")
    
    # 4. Kapasite kontrolü
    print("\nKAPASİTE KONTROLÜ:")
    kapasite_asimi_var = False
    for edge in edges:
        toplam_akis = sum(
            solution.get_value(path_flow[(s, t, k)]) * delta[(s, t, k)][edge]
            for s in baslangic_nodes
            for t in bitis_nodes
            for k in range(len(paths[(s, t)]))
            if (s, t, k) in path_flow and (s, t, k) in delta
        )
        if toplam_akis > capacity[edge] + 0.01:
            print(f"✗ Kapasite aşımı - {edge}: Akış={toplam_akis:.2f}, Kapasite={capacity[edge]}")
            kapasite_asimi_var = True
    
    if not kapasite_asimi_var:
        print("✓ Tüm kapasite kısıtları sağlandı")


def print_passenger_distribution(passengers):
    print("\n========= BAŞLANGIÇ NOKTALARI YOLCU DAĞILIMI =========")
    print("-" * 50)
    print(f"{'Başlangıç Noktası':<20} {'Yolcu Sayısı':>15}")
    print("-" * 50)
    
    toplam_yolcu = 0
    for node, count in passengers.items():
        if node.startswith('Başlangıç') and count > 0:
            print(f"{node:<20} {count:>15,}")
            toplam_yolcu += count
    
    print("-" * 50)
    print(f"{'TOPLAM':<20} {toplam_yolcu:>15,}")
    print("-" * 50)
    
    return toplam_yolcu


def run_two_stage_optimization():
    """
    İki aşamalı optimizasyon sürecini çalıştırır.
    """
    # AŞAMA 1 - Orijinal Model
    print("AŞAMA 1 - ORİJİNAL MODEL")
    print("-------------------------")
    toplam_yolcu = print_passenger_distribution(passengers)
    
    # Orijinal modeli oluştur
    (model1, paths1, path_flow1, path_time1, baslangic_nodes1, bitis_nodes1, 
     passengers1, capacity1, delta1, toplam_baslangic_yolcu1, edges1) = create_evacuation_model(
        edges, passengers, capacity, travel_time
    )
    
    # Modeli çöz
    solution1 = solve_model(model1)
    
    # Modeli çözdüysek sonuçları yazdır
    if solution1:
        print(f"\nModel 1 Çözüm Durumu: {solution1.solve_status}")
        print_results(solution1, path_flow1, paths1, path_time1, baslangic_nodes1, 
                      bitis_nodes1, passengers1, capacity1, delta1, toplam_baslangic_yolcu1, edges1)
    else:
        print("Model 1 için çözüm bulunamadı!")
    
    # AŞAMA 2 - Yeni Model
    print("\nAŞAMA 2 - YENİ MODEL")
    print("-------------------------")
    new_edges, new_travel_time, new_capacity, new_passengers = create_stage_two_network(toplam_yolcu)
    print_passenger_distribution(new_passengers)
    
    # Yeni modeli oluştur
    (model2, paths2, path_flow2, path_time2, baslangic_nodes2, bitis_nodes2, 
     passengers2, capacity2, delta2, toplam_baslangic_yolcu2, edges2) = create_evacuation_model(
        new_edges, new_passengers, new_capacity, new_travel_time
    )
    
    # Yeni modeli çöz
    solution2 = solve_model(model2)
    
    # Modeli çözdüysek sonuçları yazdır
    if solution2:
        print(f"\nModel 2 Çözüm Durumu: {solution2.solve_status}")
        print_results(solution2, path_flow2, paths2, path_time2, baslangic_nodes2, 
                      bitis_nodes2, passengers2, capacity2, delta2, toplam_baslangic_yolcu2, edges2)
    else:
        print("Model 2 için çözüm bulunamadı!")

# Modeli çalıştır
if __name__ == "__main__":
    run_two_stage_optimization()


AŞAMA 1 - ORİJİNAL MODEL
-------------------------

========= BAŞLANGIÇ NOKTALARI YOLCU DAĞILIMI =========
--------------------------------------------------
Başlangıç Noktası       Yolcu Sayısı
--------------------------------------------------
Başlangıç1                     2,000
Başlangıç2                     1,000
Başlangıç3                     2,200
Başlangıç5                     2,800
Başlangıç6                     3,000
Başlangıç8                     2,000
Başlangıç9                     3,000
--------------------------------------------------
TOPLAM                        16,000
--------------------------------------------------
Başlangıç noktalarındaki toplam yolcu: 16000

Model 1 Çözüm Durumu: JobSolveStatus.OPTIMAL_SOLUTION

SONUÇLAR:
Toplam Seyahat Süresi: 3219360.00 dakika

ÇÖZÜM DETAYLARI:
Optimality Gap (Göreceli GAP): nan
Çözüm Süresi: 0.00 saniye

Çözüm detayları alınamadı: 'SolveDetails' object has no attribute 'number_of_nodes'

YOL AKIŞLARI:
-------------------------